In [3]:
import os
import pandas as pd
import numpy as np
import pymc3 as pm
import theano.tensor as tt

In [4]:
base_dir = os.path.join("/Users", "sbussmann", "Development", "buda", "buda-ratings")
interim_dir = os.path.join(base_dir, 'data', 'interim')

In [5]:
league_id = 40264
winloss = pd.read_csv(os.path.join(interim_dir, "winloss_simpleprior_{}.csv".format(league_id)))

In [38]:
winloss.head(20)

Team A                             Team B  Team A Wins  Index A  \
0        AHOC                          Gothrilla         True        0   
1        AHOC                                BBN         True        0   
2        AHOC                       Stonecutters         True        0   
3        AHOC                          FlowChart         True        0   
4        AHOC                  Lady and the BAMF         True        0   
5        AHOC                           Swingers         True        0   
6        AHOC                           Upstream         True        0   
7        AHOC                              Tubbs         True        0   
8        AHOC                       Stonecutters         True        0   
9        AHOC                         Zerg Rush!         True        0   
10       AHOC                         Turtle Boy         True        0   
11       AHOC  Live Poultry, Fresh Killed (LPFK)         True        0   
12       AHOC                          TuneSquad         True        0   
13       AHOC                           JuJu Hex         True        0   
14  Gothrilla                           Swingers        False        1   
15  Gothrilla                          FlowChart         True        1   
16  Gothrilla              Hucky McThrowandcatch         True        1   
17  Gothrilla               SnakeCountryBromance        False        1   
18  Gothrilla                         Zerg Rush!        False        1   
19  Gothrilla                        GrassBurner         True        1   

    Index B  
0         1  
1         9  
2        41  
3         2  
4        28  
5         5  
6        10  
7        29  
8        41  
9         4  
10        7  
11        8  
12        6  
13       43  
14        5  
15        2  
16       50  
17       40  
18        4  
19        3

In [7]:
winloss.shape

(522, 5)

In [8]:
teams = set(np.append(winloss['Team A'].unique(), winloss['Team B'].unique()))

In [9]:
team1 = winloss['Index A'].values
team2 = winloss['Index B'].values

with pm.Model() as model:
    
    sharpness = pm.HalfStudentT('sharpness', sd=2.5, nu=3)
    ratings = pm.Normal('ratings', mu=0, shape=len(teams))
    
    logit = sharpness * (ratings[team1] - ratings[team2])

    p = tt.nnet.sigmoid(logit)
    win = pm.Bernoulli('win', p, observed=winloss['Team A Wins'].values)

In [10]:
with model:
    trace = pm.sample(1000)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [ratings, sharpness_log__]
100%|██████████| 1500/1500 [00:03<00:00, 457.42it/s]
The number of effective samples is smaller than 25% for some parameters.


In [11]:
meanratings = trace.get_values('ratings').mean(axis=0)

In [13]:
names = []
for i, meanrating in enumerate(meanratings):
    if i in winloss['Index A'].values:
        index = winloss['Index A'] == i
        name = winloss.loc[index, 'Team A'].unique()[0]
    else:
        index = winloss['Index B'] == i
        name = winloss.loc[index, 'Team B'].unique()[0]
    names.append(name)
#     print("{}: {:.3f}".format(name, meanrating))

In [18]:
ratingsdf = pd.DataFrame({
    'mean_rating': meanratings
}, index=names)

In [21]:
ratingsdf.sort_values('mean_rating', ascending=False)

mean_rating
AHOC                                           2.270726
SnakeCountryBromance                           1.745165
Injustice League                               1.531902
Pink Flamingos                                 1.383812
Zerg Rush!                                     1.305950
Jack's Abby HAOS Lager                         1.301404
Maverick                                       1.255374
JuJu Hex                                       1.179236
Gothrilla                                      1.159863
FlowChart                                      1.106415
Stonecutters                                   1.095992
Upstream                                       1.017672
Baba Yaga                                      1.006337
Puddingstone                                   0.977757
GrassBurner                                    0.653895
Bacon                                          0.628860
Hippos                                         0.623323
License to Kilt (fka Scoobers in Scotland)     0.619969
Flaming Croissants                             0.497124
Rubs The Duckie                                0.489880
Too Drunk to Fail                              0.448000
Hucky McThrowandcatch                          0.390461
Swingers                                       0.280830
JHOP                                           0.258844
TuneSquad                                      0.251491
Toads                                          0.232410
SHRedline                                      0.179604
Get Flat Ultimate                              0.173187
Turtle Boy                                     0.153256
Flying Salsa                                   0.116880
...                                                 ...
Booooooosh!                                   -0.042821
Oddjob                                        -0.067425
Fetch                                         -0.123215
The Funky Bunch                               -0.219294
A Lil Bit Sticky                              -0.227476
Moose Lightning                               -0.255970
BBN                                           -0.273646
Rabbit Disc                                   -0.323075
Demons                                        -0.350322
Nerd Alert                                    -0.385934
Alpha No Beta                                 -0.570593
Baboon Heart                                  -0.612036
Spam                                          -0.682285
Stack to the Future                           -0.686306
Lady and the BAMF                             -0.724238
Tofu Wolf                                     -0.778905
Batman and the Robins                         -0.794153
Underwater Monkey Cowboys                     -0.861150
Iron Squids                                   -0.914500
SIR!                                          -0.935076
Hipster Kitties                               -1.017119
Top Shelf                                     -1.026949
Jiggy                                         -1.209500
Tubbs                                         -1.229520
M.O.B.                                        -1.311766
Killer Bees                                   -1.421873
123 Trap!                                     -1.575853
THEM!                                         -1.603837
Spawning Alewives                             -1.823574
TuneSquad Squared                             -1.923398

[67 rows x 1 columns]

In [23]:
plusminus = pd.read_csv(os.path.join(interim_dir, 'plusminus_{}.csv'.format(league_id)), index_col='Team Name')

In [25]:
totaldf = plusminus.join(ratingsdf, how='inner')

In [27]:
totaldf.sort_values('mean_rating', ascending=False)

PlusMinus    divname  Wins  \
AHOC                                              100  4/3 Div 1    14   
SnakeCountryBromance                               33  5/2 Div 1     5   
Injustice League                                  100  4/3 Div 2    14   
Pink Flamingos                                     58  5/2 Div 2    10   
Zerg Rush!                                         22  4/3 Div 1    10   
Jack's Abby HAOS Lager                             46  4/3 Div 2    10   
Maverick                                           42  5/2 Div 2     9   
JuJu Hex                                           72  5/2 Div 2    11   
Gothrilla                                          47  4/3 Div 1    12   
FlowChart                                          29  4/3 Div 1    12   
Stonecutters                                       24  5/2 Div 1     8   
Upstream                                          119  4/3 Div 2    20   
Baba Yaga                                          47  5/2 Div 2     9   
Puddingstone                                       98  4/3 Div 2    18   
GrassBurner                                        29  4/3 Div 1    11   
Bacon                                              18  5/2 Div 2     7   
Hippos                                             26  5/2 Div 2     9   
License to Kilt (fka Scoobers in Scotland)         46  4/3 Div 2    13   
Flaming Croissants                                 66  4/3 Div 2    15   
Rubs The Duckie                                    30  4/3 Div 2     9   
Too Drunk to Fail                                  59  4/3 Div 3    11   
Hucky McThrowandcatch                              -1  5/2 Div 2     5   
Swingers                                           -6  4/3 Div 1     5   
JHOP                                               32  4/3 Div 2    12   
TuneSquad                                          -9  4/3 Div 1     6   
Toads                                              40  4/3 Div 2    15   
SHRedline                                          -6  5/2 Div 2     6   
Get Flat Ultimate                                   2  5/2 Div 2    10   
Turtle Boy                                        -12  4/3 Div 1     5   
Flying Salsa                                       13  5/2 Div 3    12   
...                                               ...        ...   ...   
Booooooosh!                                         0  4/3 Div 2     4   
Oddjob                                             11  4/3 Div 3    13   
Fetch                                               9  4/3 Div 2     8   
The Funky Bunch                                   -29  5/2 Div 2     4   
A Lil Bit Sticky                                  -43  4/3 Div 2     5   
Moose Lightning                                   -65  4/3 Div 2     4   
BBN                                               -47  4/3 Div 1     2   
Rabbit Disc                                        37  5/2 Div 3    12   
Demons                                             12  5/2 Div 3     5   
Nerd Alert                                        -54  4/3 Div 2     7   
Alpha No Beta                                      -7  5/2 Div 3    10   
Baboon Heart                                      -35  4/3 Div 3     5   
Spam                                              -36  5/2 Div 3     9   
Stack to the Future                                -7  4/3 Div 3     9   
Lady and the BAMF                                 -74  4/3 Div 2     5   
Tofu Wolf                                         -15  5/2 Div 3    11   
Batman and the Robins                              -3  4/3 Div 3     6   
Underwater Monkey Cowboys                         -24  4/3 Div 3     6   
Iron Squids                                       -33  5/2 Div 2     1   
SIR!                                              -53  5/2 Div 3     4   
Hipster Kitties                                   -98  4/3 Div 2     3   
Top Shelf                                         -35  4/3 Div 3     2   
Jiggy                                           

In [28]:
index_injustice = (winloss['Team A'] == 'Injustice League') | (winloss['Team B'] == 'Injustice League')

In [29]:
winloss[index_injustice]

Team A                  Team B  Team A Wins  \
66                          Zerg Rush!        Injustice League         True   
85                          Turtle Boy        Injustice League        False   
99   Live Poultry, Fresh Killed (LPFK)        Injustice League        False   
104                                BBN        Injustice League        False   
127                   Injustice League        A Lil Bit Sticky         True   
128                   Injustice League  Jack's Abby HAOS Lager         True   
129                   Injustice League         Hipster Kitties         True   
130                   Injustice League         Moose Lightning         True   
131                   Injustice League                   Tubbs         True   
132                   Injustice League             Booooooosh!         True   
133                   Injustice League                Dominion         True   
134                   Injustice League                   Fetch         True   
135                   Injustice League         Hipster Kitties         True   
136                   Injustice League         Rogues' Gallery         True   
137                   Injustice League            Puddingstone         True   

     Index A  Index B  
66         4       11  
85         7       11  
99         8       11  
104        9       11  
127       11       25  
128       11       14  
129       11       30  
130       11       27  
131       11       29  
132       11       22  
133       11       23  
134       11       21  
135       11       30  
136       11       24  
137       11       12

In [32]:
index_zerg = (winloss['Team A'] == 'Zerg Rush!') | (winloss['Team B'] == 'Zerg Rush!')

In [33]:
winloss[index_zerg]

Team A                             Team B  Team A Wins  Index A  \
9          AHOC                         Zerg Rush!         True        0   
18    Gothrilla                         Zerg Rush!        False        1   
27    Gothrilla                         Zerg Rush!         True        1   
43    FlowChart                         Zerg Rush!        False        2   
54  GrassBurner                         Zerg Rush!        False        3   
61   Zerg Rush!                          TuneSquad         True        4   
62   Zerg Rush!               SnakeCountryBromance        False        4   
63   Zerg Rush!                    Moose Lightning         True        4   
64   Zerg Rush!                           Swingers         True        4   
65   Zerg Rush!                         Turtle Boy         True        4   
66   Zerg Rush!                   Injustice League         True        4   
67   Zerg Rush!                       Puddingstone        False        4   
68   Zerg Rush!                              Fetch         True        4   
69   Zerg Rush!  Live Poultry, Fresh Killed (LPFK)         True        4   

    Index B  
9         4  
18        4  
27        4  
43        4  
54        4  
61        6  
62       40  
63       27  
64        5  
65        7  
66       11  
67       12  
68       21  
69        8

In [36]:
index_gothrilla = (winloss['Team A'] == 'Gothrilla') | (winloss['Team B'] == 'Gothrilla')

In [37]:
winloss[index_gothrilla]

Team A                             Team B  Team A Wins  Index A  \
0        AHOC                          Gothrilla         True        0   
14  Gothrilla                           Swingers        False        1   
15  Gothrilla                          FlowChart         True        1   
16  Gothrilla              Hucky McThrowandcatch         True        1   
17  Gothrilla               SnakeCountryBromance        False        1   
18  Gothrilla                         Zerg Rush!        False        1   
19  Gothrilla                        GrassBurner         True        1   
20  Gothrilla                         Turtle Boy         True        1   
21  Gothrilla  Live Poultry, Fresh Killed (LPFK)         True        1   
22  Gothrilla                           Upstream         True        1   
23  Gothrilla  Live Poultry, Fresh Killed (LPFK)         True        1   
24  Gothrilla                          TuneSquad         True        1   
25  Gothrilla                                BBN         True        1   
26  Gothrilla                           Swingers         True        1   
27  Gothrilla                         Zerg Rush!         True        1   
28  Gothrilla                        GrassBurner        False        1   
29  Gothrilla                     Shake and Bake         True        1   

    Index B  
0         1  
14        5  
15        2  
16       50  
17       40  
18        4  
19        3  
20        7  
21        8  
22       10  
23        8  
24        6  
25        9  
26        5  
27        4  
28        3  
29       42